In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_35_topics/lda_model_35_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 33: 0.098*"bruins" + 0.069*"pac" + 0.052*"trojans" + 0.036*"los" + 0.032*"sun_devils" + 0.022*"photo" + 0.017*"cal" + 0.015*"conference" + 0.014*"ducks" + 0.012*"gottlieb"
Topic 28: 0.107*"gophers" + 0.087*"minnesota" + 0.032*"ten" + 0.030*"johnson" + 0.021*"staff" + 0.015*"season" + 0.013*"team" + 0.010*"dakota" + 0.010*"north_dakota" + 0.010*"vikings"
Topic 20: 0.033*"football" + 0.031*"quarterback" + 0.021*"nfl" + 0.021*"defensive" + 0.020*"college_football" + 0.012*"receiver" + 0.012*"coordinator" + 0.011*"running" + 0.011*"offensive" + 0.009*"field"
Topic 16: 0.062*"nba" + 0.028*"basketball" + 0.014*"celtics" + 0.014*"lakers" + 0.013*"star" + 0.012*"league" + 0.011*"bulls" + 0.011*"player" + 0.010*"draft" + 0.010*"pick"
Topic 10: 0.030*"temple" + 0.029*"wright" + 0.018*"willard" + 0.015*"owls" + 0.015*"atlantic" + 0.015*"la" + 0.014*"drexel" + 0.014*"joe" + 0.013*"wildcats" + 0.013*"salle"
Topic 3: 0.052*"yards" + 0.018*"yard" + 0.018*"touchdowns" + 0.017*"harbaugh" 

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import json
import pandas as pd

# Assuming your JSON file contains a list of lists, where each inner list is a tokenized document
# Replace '/content/drive/MyDrive/path/to/your/texts.json' with the actual path to your JSON file
texts_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json'

with open(texts_file_path, 'r') as f:
    texts = json.load(f)

print(f"Loaded {len(texts)} documents from {texts_file_path}")

Loaded 149649 documents from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json


In [ ]:
# import pandas as pd

# # Load the tokenized data from the Excel file
# try:
#     tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
#     print("Excel file loaded successfully.")
#     # Assuming the original text is in a column named 'text' or similar.
#     # Please adjust the column name if it's different in your Excel file.
#     if 'Main_text_tokenized' in tokenized_data_df.columns:
#         texts = tokenized_data_df['Main_text_tokenized'].tolist()
#         print(f"Extracted {len(texts)} documents.")
#     else:
#         texts = None
#         print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

# except FileNotFoundError:
#     print("Error: The file was not found at the specified path.")
# except Exception as e:
#     print(f"An error occurred while reading the Excel file: {e}")

Excel file loaded successfully.
Extracted 149703 documents.


In [5]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.2807,"family, school, life, father, basketball, home...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.1975,"family, school, life, father, basketball, home...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,16,0.2509,"nba, basketball, celtics, lakers, star, league...","[garden, missed, lonely, nights, season, debut..."
3,3,7,0.1914,"olympic, world, olympics, won, women, time, go...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,1,0.1688,"league, draft, season, nfl, team, round, milli...","[robinson, talk, rangers, stanley, cup, finals..."
5,5,11,0.3071,"baseball, softball, tbd, innings, hit, college...","[conference, pacific, coach_mike, gillespie, s..."
6,6,1,0.4016,"league, draft, season, nfl, team, round, milli...","[rangers, signed, center, michael, nylander, m..."
7,7,31,0.2079,"goals, hockey, goal, season, cronin, ice, game...","[gavitt, elected, hall, modest, player, dartmo..."
8,8,14,0.2681,"season, guard, team, player, rebounds, forward...","[mythical, element, senior_guard, seimone, aug..."
9,9,4,0.256,"family, school, life, father, basketball, home...","[ncaa_women, jim, izard, diana, vines, remembe..."


In [6]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.2807,"family, school, life, father, basketball, home...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.1975,"family, school, life, father, basketball, home...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,16,0.2509,"nba, basketball, celtics, lakers, star, league...","[garden, missed, lonely, nights, season, debut..."
3,3,7,0.1914,"olympic, world, olympics, won, women, time, go...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,1,0.1688,"league, draft, season, nfl, team, round, milli...","[robinson, talk, rangers, stanley, cup, finals..."


In [7]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[48272, 60670, 41524, 3062, 37592, 47626, 74180, 1559, 36857, 30983, 41663, 72012, 21372, 3064, 65983, 48414, 50073, 43428, 7154, 75518]

Topic 1:
[121690, 106857, 120843, 12464, 35493, 29263, 36414, 66420, 34420, 140455, 94057, 13824, 113778, 119726, 1975, 107744, 135710, 147245, 30662, 41037]

Topic 2:
[98680, 120467, 80907, 125383, 89542, 18118, 65224, 124320, 10627, 72246, 55312, 138267, 78306, 93669, 130664, 137874, 92216, 138322, 70093, 90129]

Topic 3:
[20671, 57020, 42451, 16834, 68235, 112805, 13039, 21250, 13038, 35187, 592, 6657, 22780, 28768, 19663, 17638, 63783, 25979, 9886, 107342]

Topic 4:
[63005, 11603, 60321, 45848, 23151, 67091, 15527, 123383, 25149, 31041, 15526, 76456, 95419, 69341, 100772, 75224, 3633, 23330, 87936, 53231]

Topic 5:
[115822, 25870, 100438, 57162, 15802, 33888, 57083, 14542, 93570, 127167, 77012, 121819, 67515, 56172, 24077, 124713, 127332, 68207, 61206, 95346]

Topic 6:

In [8]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,509
1,3447
2,8475
3,4273
4,3010
5,6092
6,1068
7,8189
8,2326


In [9]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,48272,0,0.8489,"students, university, black, school, student, ...","[university, federal, graduation_rates, athlet..."
1,60670,0,0.7608,"students, university, black, school, student, ...","[ncaa, academic, reforms, refined, eligibility..."
2,41524,0,0.7004,"students, university, black, school, student, ...","[university, graduated, percent, scholarship, ..."
3,3062,0,0.6994,"students, university, black, school, student, ...","[graduation, rate, university, student_athlete..."
4,37592,0,0.6936,"students, university, black, school, student, ...","[athlete, graduation, rate, fell, students, ti..."
...,...,...,...,...,...
95,121843,0,0.4079,"students, university, black, school, student, ...","[consistently, lauded, universities, country, ..."
96,29320,0,0.407,"students, university, black, school, student, ...","[boston_college_football_team, embarks, idaho,..."
97,13135,0,0.4035,"students, university, black, school, student, ...","[profits, fun, ncaa_basketball_tournament, pla..."
98,73863,0,0.4014,"students, university, black, school, student, ...","[lowest, graduation, rate, percent, teams_ncaa..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,121690,1,0.6615,"league, draft, season, nfl, team, round, milli...","[flyers, found, center, monday, night, acquire..."
101,106857,1,0.6507,"league, draft, season, nfl, team, round, milli...","[surprisingly, sporting, director, earnie, ste..."
102,120843,1,0.6254,"league, draft, season, nfl, team, round, milli...","[jeff, gorton, enjoying, fabulous, summer, ran..."
103,12464,1,0.6146,"league, draft, season, nfl, team, round, milli...","[cal, northridge, goalkeepr, joe, barton, draf..."
104,35493,1,0.6101,"league, draft, season, nfl, team, round, milli...","[baseball, american, league, activated, dan, w..."
...,...,...,...,...,...
195,130920,1,0.4374,"league, draft, season, nfl, team, round, milli...","[phillip, danault, signed, million, contract, ..."
196,131943,1,0.4374,"league, draft, season, nfl, team, round, milli...","[major, league, soccer, expansion, club, charl..."
197,106391,1,0.4365,"league, draft, season, nfl, team, round, milli...","[secrecy, fundamental, nfl, playbook, teams, r..."
198,127942,1,0.4365,"league, draft, season, nfl, team, round, milli...","[thursday, d.c, united, found, top, mls, draft..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3059,31667,32,0.8501,"golf, round, armour, championship, shot, tour,...","[trip, masters, rewarding, mesa, resident, mar..."
3060,22333,32,0.8494,"golf, round, armour, championship, shot, tour,...","[ernie, els, ben, curtis, share, birdies, toug..."
3061,131005,32,0.8378,"golf, round, armour, championship, shot, tour,...","[valspar, championship, returns, palm, harbor,..."
3062,22335,32,0.8346,"golf, round, armour, championship, shot, tour,...","[annika, sorenstam, mad, finished, birdies, gr..."
3063,81958,32,0.8335,"golf, round, armour, championship, shot, tour,...","[defending_champion, bryce, molder, earned, pg..."
...,...,...,...,...,...
3154,131782,32,0.6784,"golf, round, armour, championship, shot, tour,...","[holes, robert, gearhart, carefree, verde, riv..."
3155,25391,32,0.6783,"golf, round, armour, championship, shot, tour,...","[european, event, world, match, play, champion..."
3156,38959,32,0.6779,"golf, round, armour, championship, shot, tour,...","[stewart, cink, swinging, thinking, round, col..."
3157,5665,32,0.6774,"golf, round, armour, championship, shot, tour,...","[golf, greg, norman, winless, pga_tour, shot, ..."



--- Topic 33 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3159,28671,33,0.6734,"bruins, pac, trojans, los, sun_devils, photo, ...","[caption, march_madness, road, st_louis, ncaa_..."
3160,111292,33,0.6332,"bruins, pac, trojans, los, sun_devils, photo, ...","[defeated, ncaa, college_football, los, memori..."
3161,44326,33,0.6255,"bruins, pac, trojans, los, sun_devils, photo, ...","[caption, text, illustration, caption, photo, ..."
3162,144214,33,0.571,"bruins, pac, trojans, los, sun_devils, photo, ...","[versus, ncaa_tournament, time, printing, edit..."
3163,87252,33,0.5658,"bruins, pac, trojans, los, sun_devils, photo, ...","[caption, text, illustration, caption, photo, ..."
...,...,...,...,...,...
3254,87335,33,0.3328,"bruins, pac, trojans, los, sun_devils, photo, ...","[urgency, word, day, play, saturday, maples, p..."
3255,15259,33,0.3322,"bruins, pac, trojans, los, sun_devils, photo, ...","[tempe, ariz, ned, wulk, led, ncaa_basketball_..."
3256,69630,33,0.3321,"bruins, pac, trojans, los, sun_devils, photo, ...","[caption, text, illustration, caption, history..."
3257,80510,33,0.3311,"bruins, pac, trojans, los, sun_devils, photo, ...","[guard, lazeric, jones, likes, environment, mc..."



--- Topic 34 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3259,96115,34,0.7714,"trump, president, people, sports_betting, gamb...","[legislature, gov, christie, bill, thursday, r..."
3260,7937,34,0.7554,"trump, president, people, sports_betting, gamb...","[sacramento, senate, voted, thursday, ban, sal..."
3261,98758,34,0.7524,"trump, president, people, sports_betting, gamb...","[federal, appeals, court, revived, quest, lega..."
3262,6504,34,0.7449,"trump, president, people, sports_betting, gamb...","[tony, batt, stephens, bureau, resurrection, b..."
3263,108686,34,0.7402,"trump, president, people, sports_betting, gamb...","[day, legal, quest, evade, federal, law, barri..."
...,...,...,...,...,...
3354,116609,34,0.594,"trump, president, people, sports_betting, gamb...","[supreme_court, ruled, monday, purge, voter, r..."
3355,49554,34,0.5926,"trump, president, people, sports_betting, gamb...","[tony, batt, stephens, bureau, sen, mccain, ap..."
3356,108656,34,0.5921,"trump, president, people, sports_betting, gamb...","[american, democracy, comic, opera, frequently..."
3357,102057,34,0.5919,"trump, president, people, sports_betting, gamb...","[ebola, vaccines, safe, trial, experimental, e..."


In [10]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_35_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_35_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**